In [1]:
import random
import time
from datetime import datetime
from IPython.display import clear_output
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
from  lxml import etree
import json 
import os
from tqdm import tqdm
import traceback
import undetected_chromedriver as uc
import traceback

In [9]:
service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
driver = webdriver.Edge(service=service)
url = "https://homedy.com/ban-can-ho-quan-dong-da-ha-noi/cdt-chung-cu-van-mieu-nguyen-thai-hoc-gan-hoan-kiem-es1882888"
driver.get(url)
bs = BeautifulSoup(driver.page_source)

page = etree.HTML(str(bs))


infors = bs.find_all("div", {"class": "product-detail-top-left"})[0]
infors = infors.find_all("h1")[0]
 
print(infors.text)


CĐT - bán chung cư Văn Miếu - Nguyễn Thái Học gần Hoàn Kiếm


In [ ]:
!pip install undetected_chromedriver

In [10]:



class crawling:
    
    def __init__(self,):
        self.home_page = "https://homedy.com/ban-nha-dat-ha-noi"
        self.root = "https://homedy.com"
        self.page = 1

    
    def get_pages(self, page_source):
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))

        links = []
        for i in page.xpath("""//*[@id="MainPage"]/div[3]/div[2]""")[0].getchildren():
            try:
                links.append(i.getchildren()[0][0].get("href"))
            except Exception as e:
                print(e,  "when get page")
        links = [link for link in links if link != None]
        return [self.root+link for link in links]
    
    def next_page(self):
        self.page += 1
        return  self.home_page + "f/p{self.page}"

    def get_id(self, page_source):
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        

        infors = bs.find_all("div", {"class": "product-info"})[0]
        infor = infors.find_all("p", {"class": "code"})
        id = infor[-1].text
        
        
        
        return id
    
    def gather(self, page_source):
        data = {}
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        
        #get title  #done
        infors = bs.find_all("div", {"class": "product-detail-top-left"})[0]
        infors = infors.find_all("h1")[0]
        
        title = infors.text
        data["Title"] = title

        #get price 
        infors = bs.find_all("div", {"class": "product-short-info"})[0]
        infors = infors.find_all("div", {"class" :"short-item"})

        price = infors[0].find_all("strong")[0].text
        data["Price"] = price
        #get area
        area = infors[1].find_all("strong")[0].text
        data["Area"] = area

        #price = ''.join(page.xpath("""/html/body/div[1]/div[5]/div[2]/div[2]/div/div[1]/div/div[1]/div/div[3]/div[1]/div[1]/strong""")[0].itertext())
        #unit = page.xpath("""/html/body/div[1]/div[5]/div[2]/div[2]/div/div[1]/div/div[1]/div/div[3]/div[1]/div[1]/strong""")[0].getchildren()[1].text
        #data["Price"] = price
        #data["Unit"] = unit

        #get address #done
        address =  page.xpath("""/html/body/div[1]/div[5]/div[2]/div[2]/div/div[1]/div/div[1]/div/div[2]/a""")[0].get("title")
        address = [i.strip() for i in address.split(",")[::-1]]
        data["Address"] = address
        

        #get general description # done
        descrip = ''.join(page.xpath("""//*[@id="readmore"]""")[0].itertext())
        data["Gen_descrip"] = descrip

        #get product attributes
        try:
            list_items = page.xpath("""/html/body/div[1]/div[5]/div[2]/div[2]/div/div[1]/div/div[3]""")[0].getchildren()
            for item in list_items:
                key, value = item.getchildren()
                data[key.text] = value.text
        except Exception as e:
            print(e, "no attributes")
        #get furniture  
        try:
            infors = bs.find_all("div", {"class": "utilities-detail furniture"})[0]
            infor = infors.find_all("div", {"class": "content"})[0]
            furs = infor.find_all("div", {"class": "title"})
            for fur in furs:
                data[fur.text] = 1
        except Exception as e:
            print(e, "has no furniture")
        # product informations
        infors = bs.find_all("div", {"class": "product-info"})[0]
        infors = infors.find_all("div")

        for infor in infors:
            label = infor.find_all("p")[0]
            code = infor.find_all("p")[1]
            data[label.text] =  code.text
        return data
    def run(self, start, num_of_pages):
        service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
        driver = webdriver.Edge(service=service)

        """ options = webdriver.ChromeOptions() 
        options.add_argument("start-maximized")
        driver = uc.Chrome(options=options) """

        url = self.home_page
        self.page = 1
        if start != 1:
            self.page = start
            url = self.home_page+f"/p{self.page}"
        if os.path.exists("data/index.json"):
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
            
        with open("log.txt", "w") as f:
            pass
        
        dataset = []
        for i in range(num_of_pages):
            num = 10
            pages = []
            while num > 0:
                try:
                    driver.get(url)
                    time.sleep(6)
                    pages = self.get_pages(driver.page_source)
                    driver.delete_all_cookies()
                    break
                except:
                    num -= 1
            if num <= 0:
                url = self.next_page()
                continue
            #driver.close()
            #service = webdriver.edge.service.Service("edgedriver_win64\msedgedriver.exe")
            #driver = webdriver.Edge(service=service)
            for page in tqdm(pages):
                num = 1
                

                while num > 0:
                    try:
                        driver.get(page)
                        time.sleep(6)

                        id = self.get_id(driver.page_source)
                        if id in index: 
                            print(id, "exist")
                            break

                        data = self.gather(driver.page_source)
                        
                        if data["ID tin"] not in index:
                            dataset.append(data)
                            index.add(data["ID tin"])
                        with open("log.txt", "a") as f:
                            f.write("Success:" + page +"\n")
                        driver.delete_all_cookies()
                        break
                    except Exception as e:
                        print(e, "unknown")
                        with open("log.txt", "a") as f:
                            f.write(f"Excution{5-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        
                        with open("failed.txt", "a") as f:
                            f.write(f"Excution{5-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        driver.delete_all_cookies()
                        num -= 1
            url = self.next_page()
        if len(dataset) != 0:
            df = pd.DataFrame(dataset)
            df.to_csv(f"data/{start}_{start+num_of_pages}_{time.time()}.csv", index=False)
        with open("data/index.json", "w") as f:
            json.dump(list(index), f)
        driver.close()
        
craw = crawling()

In [ ]:
c = crawling()

In [12]:
for i in range(1 , 2):
    craw.run(i, 1)
    clear_output()


list index out of range when get page
list index out of range when get page


 39%|███▉      | 7/18 [00:52<01:21,  7.42s/it]

list index out of range has no furniture


 44%|████▍     | 8/18 [00:59<01:13,  7.36s/it]

list index out of range has no furniture


...

list index out of range has no furniture


 89%|████████▉ | 16/18 [01:59<00:15,  7.64s/it]

list index out of range has no furniture


100%|██████████| 18/18 [02:13<00:00,  7.42s/it]


OSError: Cannot save file into a non-existent directory: 'data'